<a href="https://colab.research.google.com/github/ananya24-gif/-_-1/blob/main/AVATAAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#task1
import torch
from PIL import Image
import requests
import matplotlib.pyplot as plt
import cv2
import numpy as np
from transformers import SamProcessor, SamModel

def load_image(image_path):
    """Loads image using PIL and converts it to numpy array."""
    image = Image.open(image_path)
    return np.array(image)

def segment_object(image, class_prompt):
    """
    Uses a pre-trained segmentation model (e.g. Segment Anything or DeepLabV3)
    to generate a mask for the specified class.
    """
    # Load a pre-trained segmentation model, e.g., Segment Anything Model (SAM)
    processor = SamProcessor.from_pretrained("facebook/sam-vit-huge")
    model = SamModel.from_pretrained("facebook/sam-vit-huge")

    # Preprocess the image and create input encoding based on class prompt
    inputs = processor(text=class_prompt, images=image, return_tensors="pt")
    outputs = model(**inputs)

    # Get the segmentation mask and threshold it
    mask = outputs.logits.argmax(dim=1).squeeze().cpu().numpy()

    # Create a red mask on top of the object
    red_mask = np.zeros_like(image)
    red_mask[mask == 1] = [255, 0, 0]  # Apply red where the mask is active

    # Overlay red mask on the original image
    masked_image = cv2.addWeighted(image, 0.7, red_mask, 0.3, 0)

    return masked_image

def save_output_image(output_image, output_path):
    """Save the resulting image to the given path."""
    Image.fromarray(output_image).save(output_path)

if __name__ == "__main__":
    import argparse

    # Argument parsing
    parser = argparse.ArgumentParser(description="Object Segmentation")
    parser.add_argument("--image", type=str, required=True, help="Path to input image")
    parser.add_argument("--class", type=str, required=True, help="Class of object to segment")
    parser.add_argument("--output", type=str, required=True, help="Path to output image")

    args = parser.parse_args()

    # Load image
    image = load_image(args.image)

    # Segment object
    masked_image = segment_object(image, args.class)

    # Save output
    save_output_image(masked_image, args.output)


In [ ]:
def reposition_object(image, mask, x_shift, y_shift):
    """
    Repositions the segmented object by shifting it by x and y pixels.
    """
    # Find the bounding box of the object
    object_coords = np.where(mask == 1)
    y_min, y_max = object_coords[0].min(), object_coords[0].max()
    x_min, x_max = object_coords[1].min(), object_coords[1].max()

    # Extract the object using its bounding box
    object_image = image[y_min:y_max, x_min:x_max]

    # Create a new blank canvas for the image
    new_image = image.copy()

    # Define new position by shifting
    new_x = x_min + x_shift
    new_y = y_min + y_shift

    # Ensure the new position stays within image bounds
    new_x = max(0, min(new_x, image.shape[1] - object_image.shape[1]))
    new_y = max(0, min(new_y, image.shape[0] - object_image.shape[0]))

    # Place the object at the new position
    new_image[new_y:new_y+object_image.shape[0], new_x:new_x+object_image.shape[1]] = object_image

    return new_image

if __name__ == "__main__":
    import argparse

    # Argument parsing
    parser = argparse.ArgumentParser(description="Object Repositioning")
    parser.add_argument("--image", type=str, required=True, help="Path to input image")
    parser.add_argument("--class", type=str, required=True, help="Class of object to segment")
    parser.add_argument("--x", type=int, default=0, help="X-axis shift (positive for right, negative for left)")
    parser.add_argument("--y", type=int, default=0, help="Y-axis shift (positive for up, negative for down)")
    parser.add_argument("--output", type=str, required=True, help="Path to output image")

    args = parser.parse_args()

    # Load image and segment object
    image = load_image(args.image)
    mask = segment_object(image, args.class)  # Assuming mask is binary

    # Reposition the object
    new_image = reposition_object(image, mask, args.x, args.y)

    # Save output
    save_output_image(new_image, args.output)
